In [19]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import pandas as pd
import glob, pdb, os, sys, cv2
from ripser import ripser
from persim import plot_diagrams
import gudhi as gd
from angio_modifiedAli import angio_abm
import cv2
import numpy as np
import matplotlib.pyplot as plt


import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.morphology import skeletonize
import networkx as nx

import matplotlib.cm as cm


SyntaxError: 'return' outside function (angio_modifiedAli.py, line 357)

In [16]:
import numpy as np

def run(chemo, hapto):
    # In[ ]:
    
    #initialize simulation
    
    #### chemoattractant distribution
    # linear : profile linearly increases with x
    # tumor  : profile max as the point (1,0.5) imitating a tumor center at that location
    C_dist = 'linear'
    
    #max time units a simulation can run
    final_time = 20.0
    
    #chemotaxis rate
    chemo_rate = 'const'
    
    #how many ABM sims to run simulation
    ABM_sims = 1
    
    hapt_vec = [hapto]
    chi_vec = [chemo]
    
    psi = 0.1
    
    for i in np.arange(ABM_sims):
        
        for hapt in hapt_vec:
            for chi in chi_vec:
    
                #initialize ABM simulation
                A = angio_abm(C_dist,
                              rho = hapt,
                              t_final = final_time,
                              chi = chi,
                              chemo_rate = chemo_rate,
                              psi = psi)

                #initialize chemoattractant and TAF grids, sprout locations
                A.IC_generate()
                A.sprout_initialize()

                # Record biological data (sprouts, tips, branches)
                A.record_bio_data()
                
                # Track each vessel segment as edges in a graph
                segments = {}
                segment_id = 0
                connections = {}  # Store connections between segments
                
                #Run the ABM until either one of the sprouts reaches x=0.95
                j = 0
                max_x = 0
                while max_x < 0.95:
                    
                    # Move sprouts
                    A.move_sprouts()
                    
                    # Update TAF, chemoattractant
                    A.update_grids()
                    
                    # Perform branching
                    A.branch()
                    
                    # Save bio info
                    A.record_bio_data()
                    
                    # Track new segments created by branching
                    new_segments = A.get_new_branches()  # Hypothetical function
                    
                    for seg in new_segments:
                        start_pos = seg['start']  # Starting point of the branch
                        end_pos = seg['end']      # Current tip position
                        segments[segment_id] = {'start': start_pos, 'end': end_pos, 'length': np.linalg.norm(np.array(end_pos) - np.array(start_pos))}
                        segment_id += 1
                    
                    # Track connections between segments (anastomosis or joining)
                    joined_segments = A.get_joined_segments()  # Hypothetical function
                    for join in joined_segments:
                        parent_id = join['parent_segment']
                        child_id = join['child_segment']
                        if parent_id not in connections:
                            connections[parent_id] = []
                        connections[parent_id].append(child_id)
                    
                    # Max x value reached by sprout tips
                    max_x = np.max(A.X[A.N==1])
                    
                    j += 1
                    if A.dt * j > final_time:
                        #stop simulation if time exceeds max_sim_time
                        break

                A.save_bio_data(i)
                # Save segment and connection information as graph data
                A.save_graph_data(segments, connections)

    return A


A = run(0.4, 0.4)


NameError: name 'angio_abm' is not defined